In [1]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import awkward as ak

In [2]:
print('hi')

hi


In [3]:
file = "/home/users/nandan/tmp/CMSSW_14_1_5/src/RecoLocalTracker/SiStripClusterizer/test/test_study.root"  # replace with your ROOT file path
print(file)

/home/users/nandan/tmp/CMSSW_14_1_5/src/RecoLocalTracker/SiStripClusterizer/test/test_study.root


In [4]:
def data_loader(file_path, var, tree, signal, frac=0):
    print('calling')
    with uproot.open(file)[f'sep19_2_2_dump_raw/{tree}'] as input_tree:
         df = input_tree.arrays(var)
         df = df[(df['target']==signal)]
         df['avg_charge'] = df['charge'] / df['size']
         df['mean_x'] = ak.mean(df['x'], axis=1)
         df['mean_y'] = ak.mean(df['y'], axis=1)
         df['mean_z'] = ak.mean(df['z'], axis=1)
         std_x = ak.to_numpy(ak.std(df['x'], axis=1))
         std_x[np.isnan(std_x)] = 0
         df['std_x'] = ak.Array(std_x)
         std_y = ak.to_numpy(ak.std(df['y'], axis=1))
         std_y[np.isnan(std_y)] = 0
         df['std_y'] = ak.Array(std_y)
         std_z = ak.to_numpy(ak.std(df['z'], axis=1))
         std_z[np.isnan(std_z)] = 0
         df['std_z'] = ak.Array(std_z)
         df = pd.DataFrame({k: ak.to_list(df[k]) for k in df.fields})
         if frac:
             df = df.sample(frac=frac, random_state=42)
         return df

In [7]:
def plot_var_dist(df, varlists):
   for var in varlists:
       print(var)
       minimum_value = min(df[var])
       maximum_value = max(df[var])
       bins = np.linspace(minimum_value, maximum_value, 100)
       for label in [0,1]:
           data = df[df['target'] == label][var]
           weights = np.ones_like(data) / len(data)
           plt.hist(data, weights=weights, histtype='step', \
            label='sig' if label==1 else 'bkg', bins=bins)
       plt.legend(loc='best', title=var)
       plt.ylim([0.0001, 10])
       plt.yscale('log')
       plt.show()

In [ ]:
vars = ['size', 'charge',\
            'max_adc', 'x','y','z', \
            'max_adc_x', 'max_adc_y', 'max_adc_z', \
            'diff_adc_mone', 'diff_adc_mtwo', 'diff_adc_mthree', 'diff_adc_pone', 'diff_adc_ptwo', 'diff_adc_pthree',\
            'target', 'recotrk_dr_min']
print(vars)
print('loading sig')
sig = data_loader(file, vars, 'tree', 1, 0.001)
print('loading bg')
bkg = data_loader(file, vars, 'tree', 0, 0.001)
df = pd.concat([sig, bkg], ignore_index=True)
var[var.index('x')] = 'std_x'
var[var.index('y')] = 'std_y'
var[var.index('z')] = 'std_z'
var.insert(var.index('charge'), 'avg_charge')
plot_var_dist(df, vars)
print('plotting done')